In [ ]:
import os
import sys
import boto3
from dotenv import load_dotenv
from sagemaker.utils import name_from_base
from sagemaker.tuner import ContinuousParameter, HyperparameterTuner

sys.path.append(os.path.join("..", ".."))
from utils.estimators import BaseEstimator, BaseHyperparameterTuner

In [ ]:
load_dotenv(os.path.join("..", "..", "env"))

In [ ]:
model_id = os.environ["IMAGE_CLASSIFICATION_MODEL_ID"]
model_version = os.environ["IMAGE_CLASSIFICATION_MODEL_VERSION"]

base_job_name = name_from_base(model_id.replace("tensorflow-", ""))
print(f"base_job_name: {base_job_name}")

In [ ]:
s3_input_path = os.path.join(os.environ["S3_INPUT_BUCKET"], os.environ["S3_PREFIX"], "")
s3_output_path = os.path.join(os.environ["S3_OUTPUT_BUCKET"], os.environ["S3_PREFIX"], model_id)

print(f"s3_input_path: {s3_input_path}")
print(f"s3_output_path: {s3_output_path}")

In [ ]:
# [Optional] Override default hyperparameters with custom values
hyperparameters = {}
hyperparameters["augmentation"] = os.environ["AUGMENTATION"]
hyperparameters["augmentation_random_flip"] = os.environ["AUGMENTATION_RANDOM_FLIP"]
hyperparameters["augmentation_random_rotation"] = os.environ["AUGMENTATION_RANDOM_ROTATION"]
hyperparameters["augmentation_random_zoom"] = os.environ["AUGMENTATION_RANDOM_ZOOM"]
hyperparameters["batch_size"] = os.environ["BATCH_SIZE"]
hyperparameters["beta_1"] = os.environ["BETA_1"]
hyperparameters["beta_2"] = os.environ["BETA_2"]
hyperparameters["binary_mode"] = os.environ["BINARY_MODE"]
hyperparameters["dropout_rate"] = os.environ["DROPOUT_RATE"]
hyperparameters["early_stopping"] = os.environ["EARLY_STOPPING"]
hyperparameters["early_stopping_min_delta"] = os.environ["EARLY_STOPPING_MIN_DELTA"]
hyperparameters["early_stopping_patience"] = os.environ["EARLY_STOPPING_PATIENCE"]
hyperparameters["epochs"] = os.environ["EPOCHS"]
hyperparameters["epsilon"] = os.environ["EPSILON"]
hyperparameters["eval_metric"] = os.environ["EVAL_METRIC"]
hyperparameters["image_resize_interpolation"] = os.environ["IMAGE_RESIZE_INTERPOLATION"]
hyperparameters["initial_accumulator_value"] = os.environ["INITIAL_ACCUMULATOR_VALUE"]
hyperparameters["label_smoothing"] = os.environ["LABEL_SMOOTHING"]
hyperparameters["learning_rate"] = os.environ["LEARNING_RATE"]
hyperparameters["momentum"] = os.environ["MOMENTUM"]
hyperparameters["optimizer"] = os.environ["OPTIMIZER"]
hyperparameters["regularizers_l2"] = os.environ["REGULARIZERS_L2"]
hyperparameters["reinitialize_top_layer"] = os.environ["REINITIALIZE_TOP_LAYER"]
hyperparameters["rho"] = os.environ["RHO"]
hyperparameters["train_only_on_top_layer"] = os.environ["TRAIN_ONLY_ON_TOP_LAYER"]
hyperparameters

In [ ]:
hp_tuning = os.environ["ENABLE_HP_TUNING"] == "True"
if hp_tuning:    
    # You can select from the hyperparameters supported by the model, and configure ranges of values to be searched for training the optimal model.(https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-define-ranges.html)
    hyperparameter_ranges = {
        "learning_rate": ContinuousParameter(1e-5, 1e-1, scaling_type="Logarithmic"),
        "dropout_rate": ContinuousParameter(0.1, 0.5, scaling_type="Linear"),
        "regularizers_l2": ContinuousParameter(1e-5, 1e-1, scaling_type="Logarithmic"),
    }

In [ ]:
estimator = BaseEstimator(
    model_id=model_id,
    model_version=model_version,
    hyperparameters=hyperparameters,
    instance_type=os.environ["TRAINING_INSTANCE_TYPE"],
    instance_count=int(os.environ["TRAINING_INSTANCE_COUNT"]),
    max_run=int(os.environ["MAX_RUN"]),
    output_path=s3_output_path,
    base_job_name=base_job_name,
)
if hp_tuning:
    hp_tuner = BaseHyperparameterTuner(
        estimator=estimator,
        hyperparameter_ranges=hyperparameter_ranges,
        max_jobs=int(os.environ["MAX_JOBS"]),
        max_parallel_jobs=int(os.environ["MAX_PARALLEL_JOBS"]),
        base_job_name=base_job_name,
    )

In [ ]:
%%time
if hp_tuning:
    # Launch a SageMaker Tuning job to search for the best hyperparameters
    hp_tuner.fit({"training": s3_input_path}, logs="None")
else:
    # Launch a SageMaker Training job by passing s3 path of the training data
    estimator.fit({"training": s3_input_path}, logs="None")

In [ ]:
# Identify the previously trained model path based on the output location where artifacts are stored previously and the training job name.
if hp_tuning:  # If using amt, select the model for the best training job.
    sage_client = boto3.Session().client("sagemaker")
    tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=hp_tuner._estimator._current_job_name
    )
    last_training_job_name = tuning_job_result["BestTrainingJob"]["TrainingJobName"]
else:
    last_training_job_name = estimator._estimator._current_job_name

last_trained_model_path = f"{s3_output_path}/{last_training_job_name}/output/model.tar.gz"
print(f"Best model saved in:\n{last_trained_model_path}")